# Naive Bayes model

In our project, we expect to use Naive Bayes to train [Cornell Dataset](https://www.cs.cornell.edu/people/pabo/movie-review-data/), as comparsion to the performance of LSTM(RNN) version.

## 1 Loading Data

Load tokenized clean corpus generated by `Data_cleaning_saving.ipynb`.

In [3]:
import os

def open_file(path):
    with open(path, mode='r', errors='replace') as f:
        sentence_list = f.readlines()
    return sentence_list

pos_dir = 'data/pos_sample_tokenized.txt'
neg_dir = 'data/neg_sample_tokenized.txt'
all_dir = 'data/all_sample_tokenized.txt'

# Open text with positive sentences
pos_list = open_file(pos_dir)
# Open text with negative sentences
neg_list = open_file(neg_dir)
# Open text with all sentences
all_list = open_file(all_dir)

Build **bag of words** model for further training.

In [4]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

# Make labels for sentences
# 1: positive; 0: negative
y = np.hstack((np.ones(len(pos_list)),np.zeros(len(neg_list))))
print(f'Shape of label: {y.shape}')

# Create vectorizer
vectorizer = CountVectorizer(input='content', lowercase=False)

# Fit vectorizer with all processed tokens
# Transform them into sparse vector X
X = vectorizer.fit_transform(all_list).toarray()
print(f'Shape of vectorized dataset: {X.shape}')

Shape of label: (10655,)
Shape of vectorized dataset: (10655, 11952)


Split training set and test set.

The ratio is same as that in LSTM, which is 80% for training and 20% for testing.

In [5]:
from sklearn.model_selection import train_test_split

# Define dataset test split ratio
RANDOMNESS_SEED = 42
DATASET_TEST_SPLIT_RATIO = 0.2

Xtr, Xts, ytr, yts = train_test_split(X, y, 
                                      test_size=DATASET_TEST_SPLIT_RATIO, 
                                      random_state=RANDOMNESS_SEED, 
                                      shuffle=True)

print(f'Number of features: {Xtr.shape[1]}')
print(f'Number of training texts: {Xtr.shape[0]}')
print(f'Number of training texts: {Xts.shape[0]}')

Number of features: 11952
Number of training texts: 8524
Number of training texts: 2131


## 2 Naive Bayes Implementation

### 2.1 Feature Selection Method 1

#### 2.1.1 Feature Selection

Select 3000 words as the feature based on the frequency it appears in the **overall data set**.

In [6]:
# Select 3000 words
feature_num_1 = 3000

# Calculate frequency of each words in the overall dataset
frequency = np.sum(X, axis=0)

# Find the indices of the most frequent words
indices_1 = np.argsort(frequency)[-1:(-1*feature_num_1-1):-1]

# Show 20 selected features
print(f'First 20 selected features in total of {feature_num_1} features:')
j = 0 # List index
for i in indices_1[:20]:
    j += 1
    print(f'%02d. {vectorizer.get_feature_names()[i]}' % j)

First 20 selected features in total of 3000 features:
01. film
02. movi
03. like
04. one
05. make
06. stori
07. charact
08. time
09. comedi
10. good
11. even
12. much
13. work
14. perform
15. feel
16. way
17. get
18. littl
19. look
20. love


#### 2.1.2 Training with Bernoulli Naive Bayes

In `classifier_1_Bernoulli`:
- 1: 3000 most-frequent words, recorded as `indices_1`
- Bernoulli: Bernoulli Navie Bayes

In [7]:
from sklearn.naive_bayes import BernoulliNB

# Training and testing data with only the selected feature
Xtr_1 = Xtr[:, indices_1]
Xts_1 = Xts[:, indices_1]

# Create classifier
classifier_1_Bernoulli = BernoulliNB(alpha=1.0, binarize=0.5)

# Train the classifier
classifier_1_Bernoulli.fit(Xtr_1, ytr)

BernoulliNB(binarize=0.5)

Show the performance.

In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

def report_performance(feature_description: str, yts, ypred):
    print(f'Accuracy, precision and recall score for {feature_description} features:')
    print(f'Accuracy:\t{accuracy_score(yts, ypred)}')
    print(f'Precision:\t{precision_score(yts, ypred)}')
    print(f'Recall:\t\t{recall_score(yts, ypred)}')

# Make prediction
yhat_1_Bernoulli = classifier_1_Bernoulli.predict(Xts_1)

# Show performance
report_performance(f'{feature_num_1} most frequent', yts, yhat_1_Bernoulli)

Accuracy, precision and recall score for 3000 most frequent features:
Accuracy:	0.7583294228061943
Precision:	0.7675312199807877
Recall:		0.7453358208955224


#### 2.1.3 Training with Multinomial Naive Bayes

In [9]:
from sklearn.naive_bayes import MultinomialNB

# Create classifier
classifier_1_Multinomial = MultinomialNB(alpha=1.0)

# Train the classifier
classifier_1_Multinomial.fit(Xtr_1, ytr)

MultinomialNB()

Show the performance.

In [10]:
# Make prediction
yhat_1_Multinomial = classifier_1_Multinomial.predict(Xts_1)

# Show performance
report_performance(f'{feature_num_1} most frequent', yts, yhat_1_Multinomial)

Accuracy, precision and recall score for 3000 most frequent features:
Accuracy:	0.7573908962928203
Precision:	0.7610536218250236
Recall:		0.7546641791044776


### 2.2 Feature Selection Method 2

Instead of using term frequency, this method selects feature based on Information Gain (IG).

#### 2.2.1 Information Gain Feature Selection

Information gain measures the number of bits of information that the presence/absence of a term contributes to the attitude of review (positive/negative).

$$IG(C,X_i) = H(C) - H(C|X_i)$$

Where:

* $X_i = 0\ or\ 1$: random variable that represents the occurance of frequency of term i.

    In this project, $X_i=1$ means presence while $X_i=0$ means absence.

* $C = +\ or\ -$: random variable that determines if a review is positive or negative.

    In this project, positive reviews $C=+$ are labeled as 1, while the negative reviews $C=-$ are labeled as 0.

* $H(C)$: Inherent uncertainity (entropy) of random variable, measured in bits.

* $H(C|X_i)$: Uncertainity (entropy) given feature $X_i$ exists or not.

* $IG(C,X_i)$: Information gain, how much information does $X_i$ provide about $C$.

##### A. Inherent Uncertainity

Calculation of inherent uncertainity $H(C)$:

$$H(C) = -\sum\limits_{c\in\{+,-\}}P(C=c)log(P(C=c))$$

$$=-P(C=+)log_2(P(C=+))-P(C=-)log_2(P(C=-))$$

In [11]:
# P(pos): probability of positive review in training set
p_pos = np.sum(ytr).astype(int) / Xtr.shape[0]

# P(neg): probability of negative review in training set
p_neg = 1 - p_pos

# Inherent uncertainity
HC = - p_pos * np.log2(p_pos) - p_neg * np.log2(p_neg)
print(f'Inherent Uncertainty H(C) = {HC}')

Inherent Uncertainty H(C) = 0.9999980541295279


##### B. Conditional Entropy: Uncertainty given Specific Words

Calculation of conditional entropy $H(C|X_i)$:

$$H(C|X_i) = \sum\limits_{x}P(X_i = x)H(C|X=x)$$

$$=-\sum\limits_{c\in\{+,-\}}\sum\limits_{x\in\{0,1\}}P(X_i=x, C=c)log(P(C=c|X_i=x))$$

Where the joint probability:

$$P(X_i=x, C=c)=P(X_i=x|C=c)P(C=c)$$

and the conditional probability:

$$P(C=x|X_i=x)=\frac{P(X_i=x|C=c)P(C=c)}{P(X_i=x)}=\frac{P(X_i=x, C=c)}{P(X_i=x)}$$


P.S. Feature selection based on the IG metric only accounts for the occurrence of (not frequency with which terms appear) in the dataset.

In [12]:
import copy

# Convert matrix from term frequency to binary features
def binarization(X):
    # Copy the original data
    X_b = copy.deepcopy(X)
    # Define threshold, upper bound and lower bound
    threshold, upper, lower = 0.5, 1, 0
    # Binarization process
    X_b[X_b > threshold] = upper
    X_b[X_b <= threshold] = lower

    return X_b

# Reform document-term matrix to one-hot code
# Xtr_b: binarizied matrix of training set Xtr
Xtr_b = binarization(Xtr)

For each words X_i, count the times of being selected in positive / negative review.

Then calculate $P(X_i=x|C=c)$.

P.S.

Laplacian Smoothing is a technique that add 1 to the value of numerator and add 2 to the value of denominator.

The purpose of Laplacian Smoothing is to eliminate the effect of 0 (i.e. a word does not appear in the given sentence) in likelihood estimation. Otherwise, once there is a missing word, the whole likelihood estimation will obtain the result of 0.

In [13]:
# Boolean array indicating which sample is a positive/negative review in Xtr
pos_samples = ytr.astype(bool)
neg_samples = np.invert(pos_samples)

# Number of positive and negative reviews in Xtr
n_pos = np.sum(ytr).astype(int)
n_neg = ytr.shape[0] - n_pos

# Num{X_i = 1 | C = +}: X_i present in a positive review
n_pre_pos = np.sum(Xtr_b[pos_samples, :], axis=0)
# P(X_i = 1 | C = +)
p_pre_pos = (n_pre_pos + 1) / (n_pos + 2)  # Laplacian Smoothing

# Num{X_i = 1 | C = -}: X_i present in a negative review
n_pre_neg = np.sum(Xtr_b[neg_samples, :], axis=0)
# P(X_i = 1 | C = -)
p_pre_neg = (n_pre_neg + 1) / (n_neg + 2)  # Laplacian Smoothing

# Num{X_i = 0 | C = +}: X_i absent in a positive review
n_abs_pos = n_pos - n_pre_pos
# P(X_i = 0 | C = +)
p_abs_pos = (n_abs_pos + 1) / (n_pos + 2)  # Laplacian Smoothing

# Num{X_i = 0 | C = -}: X_i absent in a negative review
n_abs_neg = n_neg - n_pre_neg
# P(X_i = 0 | C = -)
p_abs_neg = (n_abs_neg + 1) / (n_neg + 2)  # Laplacian Smoothing

Calculate probability of positive/negative reviews $P(C=c)$.

In [14]:
# P(C = +)
p_pos = n_pos / Xtr.shape[0]
# P(C = -)
p_neg = n_neg / Xtr.shape[0]

Calculating joint probability $P(X_i=x, C=c)=P(X_i=x|C=c)P(C=c)$.

In [15]:
# P(X_i = 1, C = +)
pj_pre_pos = p_pre_pos * p_pos
# P(X_i = 1, C = -)
pj_pre_neg = p_pre_neg * p_neg
# P(X_i = 0, C = +)
pj_abs_pos = p_abs_pos * p_pos
# P(X_i = 0, C = -)
pj_abs_neg = p_abs_neg * p_neg

Probability of each words being present/absent $P(X_i=x)$.

In [16]:
n_pre = np.sum(Xtr_b, axis=0)

# P(X_i = 1)
p_pre = (n_pre + 1) / (Xtr.shape[0] + 2) # Laplacian Smoothing

# P(X_i = 0)
p_abs = 1 - p_pre

Calculating entropy segments and sum up. 

In [17]:
H1 = - pj_pre_pos * np.log2(pj_pre_pos / p_pre)
H2 = - pj_pre_neg * np.log2(pj_pre_neg / p_pre)
H3 = - pj_abs_pos * np.log2(pj_abs_pos / p_abs)
H4 = - pj_abs_neg * np.log2(pj_abs_neg / p_abs)

HC_conditional = H1 + H2 + H3 + H4

##### C. Information Gain

Calculation of information gain $IG(C,X_i)$:

$$IG(C,X_i) = H(C) - H(C|X_i)$$

Then store IG values in a text file.

In [27]:
IG = HC - HC_conditional

with open('data/IG.txt', 'w') as ig:
    for data in IG:
        ig.write(data.astype(str) + '\n')

Select 3000 words as the feature based on the information gain (IG) it possesses in **training set**.

In [19]:
# Select 3000 words
feature_num_2 = 3000

# By-default, the np.argsort will follow ASCENDING order
# Thus, we need to slice the array with negative step
indices_2 = np.argsort(IG)[-1:(-1*feature_num_2-1):-1]

# Show 20 selected features
print(f'First 20 selected features in total of {feature_num_2} features:')
j = 0 # List index
for i in indices_2[:20]:
    j += 1
    print(f'%02d. {vectorizer.get_feature_names()[i]}' % j)

First 20 selected features in total of 3000 features:
01. bad
02. bore
03. beauti
04. dull
05. perform
06. wast
07. joke
08. movi
09. heart
10. move
11. best
12. human
13. cultur
14. titl
15. film
16. examin
17. flat
18. entertain
19. unfunni
20. funni


#### 2.2.2 Training with Bernoulli Naive Bayes

In `classifier_2_Bernoulli`:
- 2: words with largest IG in descending order, recorded as `indices_2`
- Bernoulli: Bernoulli Navie Bayes

In [20]:
# Training and testing data with only the selected feature
Xtr_2 = Xtr[:, indices_2]
Xts_2 = Xts[:, indices_2]

# Create classifier
classifier_2_Bernoulli = BernoulliNB(alpha=1.0, binarize=0.5)

# Train the classifier
classifier_2_Bernoulli.fit(Xtr_2, ytr)

BernoulliNB(binarize=0.5)

Show performance.

In [21]:
# Make prediction
yhat_2_Bernoulli = classifier_2_Bernoulli.predict(Xts_2)

# Show performance
report_performance(f'{feature_num_2} with most IG', yts, yhat_2_Bernoulli)

Accuracy, precision and recall score for 3000 with most IG features:
Accuracy:	0.7611450023463163
Precision:	0.7740993184031159
Recall:		0.7416044776119403


#### 2.2.3 Training with Multinomial Naive Bayes

In [22]:
# Create classifier
classifier_2_Multinomial = MultinomialNB(alpha=1.0)

# Train the classifier
classifier_2_Multinomial.fit(Xtr_2, ytr)

MultinomialNB()

Show performance.

In [23]:
# Make prediction
yhat_2_Multinomial = classifier_2_Multinomial.predict(Xts_2)

# Show performance
report_performance(f'{feature_num_2} with most IG', yts, yhat_2_Multinomial)

Accuracy, precision and recall score for 3000 with most IG features:
Accuracy:	0.7620835288596903
Precision:	0.7657572906867357
Recall:		0.7593283582089553
